all the imports

In [1]:
import arxiv
import requests
import json

In [2]:
def get_arxiv_papers(query: str, max_papers: int = 5) -> list[dict]:
    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results = max_papers)
    papers = []

    for result in client.results(search):
        paper = {
            "title" : result.title,
            "authors": [result.author.name],
            "summary": result.summary,
            "published": result.published.date(),
            "url": result.pdf_url
        }
        papers.append(paper)

    return papers

In [6]:
from tenacity import retry, stop_after_attempt

@retry(stop=stop_after_attempt(5))


def get_ieee_papers(query:str, api_key:str, max_results:int = 5) -> list[dict]:
    url = "https://ieeexploreapi.ieee.org/api/v1/search/articles?parameter&apikey="
    params = {
        "query": query,
        'apikey': api_key,
        'max_records': max_results,
        'format': 'json'
    }

    response = requests.get(url, params=params, timeout=10)
    if response.status_code != 200:
        raise Exception(f'IEEE Xplore API failed: {response.text}')
    
    papers = []

    for article in response.json().get('articles', []):
        papers.append({
            "title": article.get('title', 'no title'),
            "authors": [author["full_name"] for author in article.get('authors', [])],
            "abstract":article.get('abstract', 'No abstract'),
            'published': article.get('publication_date',""),
            'url': article.get('pdf_url', article.get('html_url',''))
        })
    return papers

    

In [8]:
def research_agent(query:str, ieee_api_key:str, max_papers: int = 5)-> list[str]:
    try:
        papers = get_arxiv_papers(query, max_papers)
        if not papers:
            return ValueError('arXiv returned no papers')
        return papers
    except Exception as e:
        if not ieee_api_key:
            return ValueError('Switching to IEEE - Need the API key')
        print(f'arxiv failed ({e}), trying IEEE Xplore')
        return get_ieee_papers(query, ieee_api_key, max_papers)



In [9]:
def get_results(papers: list[dict]):
    for i, paper in enumerate(papers,1):
        print(f"""
        [{i}] {paper['title']}
        Authors:{','.join(paper.get('authors', []))}
        Published: {paper.get('published', 'N/A')}
        Summary: {paper.get('summary', paper.get('abstract', 'N/A'))[:200]}...
        URL:{paper.get('url','N/A')}
        """)

In [44]:
papers = research_agent('LLM quantization', ieee_api_key="#")
get_results(papers)


        [1] A Comprehensive Evaluation of Quantization Strategies for Large Language Models
        Authors:Renren Jin,Jiangcun Du,Wuwei Huang,Wei Liu,Jian Luan,Bin Wang,Deyi Xiong
        Published: 2024-02-26
        Summary: Increasing the number of parameters in large language models (LLMs) usually
improves performance in downstream tasks but raises compute and memory costs,
making deployment difficult in resource-limite...
        URL:http://arxiv.org/pdf/2402.16775v2
        

        [2] CrossQuant: A Post-Training Quantization Method with Smaller Quantization Kernel for Precise Large Language Model Compression
        Authors:Wenyuan Liu,Xindian Ma,Peng Zhang,Yan Wang
        Published: 2024-10-10
        Summary: Post-Training Quantization (PTQ) is an effective technique for compressing
Large Language Models (LLMs). While many studies focus on quantizing both
weights and activations, it is still a challenge to...
        URL:http://arxiv.org/pdf/2410.07505v1
        

       

In [45]:
print('Testing arXiv:')
arxiv_papers = get_arxiv_papers('quantum computing', max_papers=2)
get_results(arxiv_papers)

ieee_api_key = "#"
if ieee_api_key:
    print('\nTesting IEEE:')
    ieee_papers = get_ieee_papers('quantum computing', ieee_api_key, max_results=2)
    get_results(ieee_papers)

Testing arXiv:

        [1] The Rise of Quantum Internet Computing
        Authors:Seng W. Loke
        Published: 2022-08-01
        Summary: This article highlights quantum Internet computing as referring to
distributed quantum computing over the quantum Internet, analogous to
(classical) Internet computing involving (classical) distribute...
        URL:http://arxiv.org/pdf/2208.00733v1
        

        [2] Unconventional Quantum Computing Devices
        Authors:Seth Lloyd
        Published: 2000-03-31
        Summary: This paper investigates a variety of unconventional quantum computation
devices, including fermionic quantum computers and computers that exploit
nonlinear quantum mechanics. It is shown that unconven...
        URL:http://arxiv.org/pdf/quant-ph/0003151v1
        

Testing IEEE:


RetryError: RetryError[<Future at 0x23f31943e50 state=finished raised Exception>]